In [ ]:
!pip install control -U
!pip install sympy -U

In [ ]:
import control as ctrl

In [ ]:
# Numeric values of partial derivatives of theta and I_dot calculated in system_derivation.ipynb
A = 209.109726489593
B = -16.0791589363018
C = 662.228074503118
D = 0
E = -15040.865825401
F = 6.83675719336409
s = ctrl.TransferFunction.s

Gx = (C * F) / ((s ** 3) - s ** 2 * (B + E) - s * (A - B * E) + A * E)
Gx